In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import wandb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import joblib
import json
import os

In [3]:
df = pd.read_csv('./data/df_for_ML_new.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df.drop('price_per_sqft', axis=1, inplace=True)

In [5]:
# df['year_built'].fillna(-1, inplace=True)
df.fillna(-1, inplace=True)
df['year_built'] = df['year_built'].astype(int)

In [6]:
X = df.drop('target', axis=1)
y = df['target']

In [7]:
X

,baths,sqft,beds,lat,lon,pool,property_type_condo,property_type_land,property_type_multi_family,property_type_other,property_type_ranch,property_type_single_family,property_type_townhouse,property_type_traditional,state,year_built,remodeled_year,avg_school_rating,schools_qty,avg_school_dist
0,3.5,2900.0,4,35.627887,-77.335173,-1.0,0,0,0,0,0,1,0,0,37,2019,0,3.25,8.0,5.54
1,3.0,1947.0,3,47.699482,-117.190979,-1.0,0,0,0,0,0,1,0,0,53,2019,0,0.27,3.0,1.33
2,2.0,3000.0,3,33.943243,-117.376310,1.0,0,0,0,0,0,1,0,0,6,1961,1967,0.67,3.0,1.96
3,8.0,6457.0,5,32.702323,-97.347071,-1.0,0,0,0,0,0,1,0,0,48,2006,2006,0.92,4.0,0.75
4,-1.0,-1.0,1,27.041567,-82.136146,-1.0,0,1,0,0,0,0,0,0,12,0,0,0.47,3.0,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214090,3.0,2505.0,5,41.456416,-81.836809,-1.0,0,0,0,0,0,1,0,0,39,1950,2010,0.57,3.0,1.42
214091,-1.0,950.0,2,29.441120,-98.487583,1.0,0,0,0,0,0,1,0,0,48,1938,1980,0.40,3.0,1.31
214092,3.0,1792.0,4,29.826059,-95.531518,-1.0,0,0,0,0,0,1,0,0,48,1970,1970,0.27,3.0,0.66
214093,-1.0,1895.0,1,32.711046,-97.330033,-1.0,0,0,0,0,0,1,0,0,48,1921,0,5.00,3.0,1.27


In [8]:
with open('/home/dwarf/diplom/wandb.json', 'r') as f:
  data = json.load(f)
  os.environ['WANDB_API_KEY'] = data["key"]

wandb.login()
run = wandb.init(entity=None, project="diplom1", name = 'knn_run')

# Разделяем на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем модель k-NN с параметром k = 3
model = KNeighborsRegressor(n_neighbors=3)

# Обучаем модель на обучающем наборе данных
model.fit(X_train, y_train)

# Предсказываем стоимости на тестовом наборе данных
y_pred = model.predict(X_test)

# Рассчитываем метрики качества
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Логирование метрик в W&B
wandb.log({"MAE": mae})
wandb.log({"MSE": mse})
wandb.log({"R2": r2})

print("Средняя абсолютная ошибка (MAE):", mae)
print("Среднеквадратичная ошибка (MSE):", mse)
print("Коэффициент детерминации (R²):", r2)

# Сохранение модели
model_filename = "random_forest_model.pkl"
joblib.dump(model, model_filename)

# Загрузка сохраненной модели в W&B
wandb.save(model_filename)

# Завершение сессии W&B
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Средняя абсолютная ошибка (MAE): 360333.7668480504
Среднеквадратичная ошибка (MSE): 2488040522213.569
Коэффициент детерминации (R²): 0.28438565381784287


MAE,▁
MSE,▁
R2,▁
MAE,360333.76685
MSE,2488040522213.569
R2,0.28439


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

with open('/home/dwarf/diplom/wandb.json', 'r') as f:
  data = json.load(f)
  os.environ['WANDB_API_KEY'] = data["key"]

wandb.login()

# n_estimators = [50, 100, 150, 200]
n_estimators = [150]
# max_depths = [5, 10, 15, 20, 25, 30]
max_depths = [30]

for estimators in n_estimators:
    for depth in max_depths:
      run = wandb.init(entity=None, project="diplom1", name = f'rand_for_reg est={estimators} depth={depth}')

      # Создание и обучение модели случайного леса для регрессии
      model = RandomForestRegressor(n_estimators=estimators, max_depth=depth, random_state=42)

      model.fit(X_train, y_train)

      # Предсказание на тестовой выборке
      y_pred = model.predict(X_test)

      # Рассчитываем метрики качества
      mae = mean_absolute_error(y_test, y_pred)
      mse = mean_squared_error(y_test, y_pred)
      r2 = r2_score(y_test, y_pred)

      # Логирование метрик в W&B
      wandb.log({"MAE": mae})
      wandb.log({"MSE": mse})
      wandb.log({"R2": r2})   

      # Сохранение модели
      model_filename = "random_forest_model.pkl"
      joblib.dump(model, model_filename)

      # Загрузка сохраненной модели в W&B
      wandb.save(model_filename)

# Завершение сессии W&B
wandb.finish()
# print("Средняя абсолютная ошибка (MAE):", mae)
# print("Среднеквадратичная ошибка (MSE):", mse)
# print("Коэффициент детерминации (R²):", r2)




In [ ]:
with open('/home/dwarf/diplom/wandb.json', 'r') as f:
  data = json.load(f)
  os.environ['WANDB_API_KEY'] = data["key"]

wandb.login()

run = wandb.init(entity=None, project="diplom", name = f'linear_regression')

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

# Рассчитываем метрики качества
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Логирование метрик в W&B
wandb.log({"MAE": mae})
wandb.log({"MSE": mse})
wandb.log({"R2": r2})   

# Сохранение модели
model_filename = "linear_regerssion.pkl"
joblib.dump(model, model_filename)
scaler_filename = 'min_max_scaler.pkl'
joblib.dump(scaler, scaler_filename)

# Загрузка сохраненной модели в W&B
wandb.save(model_filename)
wandb.save(scaler_filename)

# Завершение сессии W&B
wandb.finish()

In [ ]:
with open('/home/dwarf/diplom/wandb.json', 'r') as f:
  data = json.load(f)
  os.environ['WANDB_API_KEY'] = data["key"]

wandb.login()

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

max_depths = [5, 10, 15, 20, 25, 30]

for depth in max_depths:
  run = wandb.init(entity=None, project="diplom", name = f'Decision Tree depth={depth}')

  # Создание и обучение модели дерева решений
  model = DecisionTreeRegressor(random_state=42)
  model.fit(X_train, y_train)

  # Предсказание на тестовом наборе
  y_pred = model.predict(X_test)

  # Рассчитываем метрики качества
  mae = mean_absolute_error(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  # Логирование метрик в W&B
  wandb.log({"MAE": mae})
  wandb.log({"MSE": mse})
  wandb.log({"R2": r2})   

  # Сохранение модели
  model_filename = "decision_tree_model.pkl"
  joblib.dump(model, model_filename)

  # Загрузка сохраненной модели в W&B
  wandb.save(model_filename)

# Завершение сессии W&B
wandb.finish()